In [7]:
import sys
sys.path.append("..")

import itertools

import torch
import numpy as np

from lra import temp_seed, ParityDataset, BinaryMarkovDataset, MajorityDataset, ListOpsTiny, IMDB

TypeError: 'NoneType' object is not an iterator

In [8]:
dataset = IMDB('imdb')
dataset.setup()
next(iter(dataset.train_dataloader(batch_size=8)))

IMDB char level | min_freq 15 | vocab size 134


(tensor([[52,  7,  3,  ...,  0,  0,  0],
         [29,  3,  9,  ...,  0,  0,  0],
         [46,  8, 13,  ...,  0,  0,  0],
         ...,
         [29,  3, 44,  ...,  0,  0,  0],
         [36, 23, 23,  ..., 19, 24,  2],
         [30, 12,  8,  ...,  0,  0,  0]]),
 tensor([0, 0, 1, 0, 1, 0, 0, 0]),
 {'lengths': tensor([ 699, 1375, 1197,  520, 2348, 1721, 2373,  732])})

In [9]:
lengths = [batch[2]['lengths'] for batch in dataset.train_dataloader(batch_size=32)]

In [10]:
lengths = torch.hstack(lengths)

In [11]:
lengths.float().mean()

tensor(1299.4161)

In [12]:
lengths.float().max()

tensor(4096.)